In [10]:
import pandas as pd
import numpy as np

In [11]:
train = pd.read_csv("../../data/raw/train_2.csv")
test = pd.read_csv("../../data/raw/test_2.csv")

In [12]:
train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SalePrice
0,11.692623,11.686189,0.0,0.730463,1.540963,1.820334,1.540963,11.170327,0.0,1.194318,...,0,0,0,0,0,0,0,0,1,12.247699
1,12.792276,0.000000,0.0,0.730463,1.540963,1.820334,0.730463,12.062832,0.0,0.000000,...,0,0,0,0,0,0,0,0,1,12.109016
2,11.892039,11.724598,0.0,0.730463,1.540963,1.820334,1.194318,10.200343,0.0,1.194318,...,0,0,0,0,0,0,0,0,1,12.317171
3,12.013683,11.354094,0.0,0.730463,1.540963,0.730463,1.540963,8.274266,0.0,0.000000,...,0,0,0,0,0,0,0,0,1,11.849405
4,12.510588,12.271365,0.0,0.730463,1.820334,1.820334,0.000000,10.971129,0.0,1.194318,...,0,0,0,0,0,0,0,0,1,12.429220


In [13]:
train_y = train['SalePrice']
train_x = train.drop(['SalePrice'], axis=1)

In [14]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# tensorflowの警告抑制
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# -----------------------------------
# ニューラルネットの実装
# -----------------------------------
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.preprocessing import StandardScaler

# データのスケーリング
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.transform(va_x)
test = scaler.transform(test)

# ニューラルネットモデルの構築
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(train_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
batch_size = 128
epochs = 100
history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y))

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = mean_squared_error(va_y, va_pred)
print(f'mean_absolute_error: {score:.4f}')

# 予測
pred = model.predict(test_x)

# -----------------------------------
# アーリーストッピング
# -----------------------------------
from keras.callbacks import EarlyStopping

# アーリーストッピングの観察するroundを20とする
# restore_best_weightsを設定することで、最適なエポックでのモデルを使用する
epochs = 100
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])

Train on 1093 samples, validate on 365 samples
Epoch 1/100
1093/1093 [==============================] - 1s 902us/step - loss: 9.1411 - mean_absolute_error: 9.1411 - val_loss: 5.5707 - val_mean_absolute_error: 5.5707
Epoch 2/100
1093/1093 [==============================] - 0s 71us/step - loss: 3.7876 - mean_absolute_error: 3.7876 - val_loss: 3.2461 - val_mean_absolute_error: 3.2461
Epoch 3/100
1093/1093 [==============================] - 0s 70us/step - loss: 2.4089 - mean_absolute_error: 2.4089 - val_loss: 1.6916 - val_mean_absolute_error: 1.6916
Epoch 4/100
1093/1093 [==============================] - 0s 72us/step - loss: 1.5962 - mean_absolute_error: 1.5962 - val_loss: 1.2100 - val_mean_absolute_error: 1.2100
Epoch 5/100
1093/1093 [==============================] - 0s 70us/step - loss: 1.3600 - mean_absolute_error: 1.3600 - val_loss: 1.0248 - val_mean_absolute_error: 1.0248
Epoch 6/100
1093/1093 [==============================] - 0s 71us/step - loss: 1.1197 - mean_absolute_error: 1.11

NameError: name 'test_x' is not defined

In [16]:
pred = model.predict(test)

In [17]:
np.expm1(pred)

array([[ 88390.25],
       [205928.98],
       [101749.03],
       ...,
       [258149.73],
       [ 76431.08],
       [242722.5 ]], dtype=float32)

In [9]:
len(train.columns)

221